###Підключення необхідних бібліотек та завантаження даних

In [3]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [4]:
# імпорт CSV file
data = pd.read_csv("data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


#АВ тести загалом по метрикам


###Обробка даних з метою групування за метриками та тестами

In [5]:
# Фільтрую дані за необхідними метриками
filtered_data = data[
    (data["event_name"].isin(["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"])) &
    (data["test_group"].isin([1, 2])) &
    (data["test"].isin([1, 2, 3, 4]))]

# Групую відфільтровані дані та рахую суму значень
grouped_data = (filtered_data.groupby(["event_name", "test_group", "test"], as_index=False)["value"].sum())

# Додаю новий стовпчик з назвою метрик
grouped_data["metric"] = grouped_data["event_name"] + "/session"

grouped_data.head()


,event_name,test_group,test,value,metric
0,add_payment_info,1,1,1988,add_payment_info/session
1,add_payment_info,1,2,2344,add_payment_info/session
2,add_payment_info,1,3,3623,add_payment_info/session
3,add_payment_info,1,4,3731,add_payment_info/session
4,add_payment_info,2,1,2229,add_payment_info/session


###Cтворення зведеної таблиці для подальших розрахунків

In [6]:
# Створюю зведену таблицю, щоб згрупувати значення за тестами
pivot_table = grouped_data.pivot(
    index= ["event_name", "metric", "test"],
    columns="test_group",
    values="value").reset_index()

# виводжу кількість сессій у розрізі тестів та груп
session_data = data[data["event_name"] == "session"]
total_sessions_by_group = session_data.groupby(["test", "test_group"], as_index=False)["value"].sum()

# додаю у таблицю нові колонки із загальною сумою сесій
pivot_table = pivot_table.merge(total_sessions_by_group[total_sessions_by_group["test_group"] == 1][["test", "value"]], on="test", how="left").rename(columns={"value": "denominator_tg_1"})
pivot_table = pivot_table.merge(total_sessions_by_group[total_sessions_by_group["test_group"] == 2][["test", "value"]], on="test", how="left").rename(columns={"value": "denominator_tg_2"})

# змінюю назви стовпців
pivot_table.rename(columns={1: "numerator_tg_1", 2: "numerator_tg_2"}, inplace=True)

pivot_table.head(10)

,event_name,metric,test,numerator_tg_1,numerator_tg_2,denominator_tg_1,denominator_tg_2
0,add_payment_info,add_payment_info/session,1,1988,2229,45362,45193
1,add_payment_info,add_payment_info/session,2,2344,2409,50637,50244
2,add_payment_info,add_payment_info/session,3,3623,3697,70047,70439
3,add_payment_info,add_payment_info/session,4,3731,3601,105079,105141
4,add_shipping_info,add_shipping_info/session,1,3034,3221,45362,45193
5,add_shipping_info,add_shipping_info/session,2,3480,3510,50637,50244
6,add_shipping_info,add_shipping_info/session,3,5298,5188,70047,70439
7,add_shipping_info,add_shipping_info/session,4,5128,4956,105079,105141
8,begin_checkout,begin_checkout/session,1,3784,4021,45362,45193
9,begin_checkout,begin_checkout/session,2,4262,4313,50637,50244


###Розрахунок необхідних показників та створення фінальної таблиці

In [7]:
# Додаю розрахунок основних показників
pivot_table["conversation_rate_tg_1"] = pivot_table["numerator_tg_1"]/pivot_table["denominator_tg_1"]
pivot_table["conversation_rate_tg_2"] = pivot_table["numerator_tg_2"]/pivot_table["denominator_tg_2"]
pivot_table["metric_change"] = (pivot_table["conversation_rate_tg_2"]/pivot_table["conversation_rate_tg_1"] - 1) * 100

# Створюю пусті списки
z_statistics = []
p_values = []
significance = []

# Роблю ітерацію за рядками
for _, row in pivot_table.iterrows():
    successes = [row["numerator_tg_1"], row["numerator_tg_2"]]
    sample_sizes = [row["denominator_tg_1"], row["denominator_tg_2"]]

# Створюю z-тест
    z_stat, p_value = proportions_ztest(count=successes, nobs=sample_sizes)

# додаю результати в відповідні списки
    z_statistics.append(z_stat)
    p_values.append(p_value)
    significance.append(p_value < 0.05)

# додаю результати в фінальну таблицю
pivot_table["z_stat"] = z_statistics
pivot_table["p_value"] = p_values
pivot_table["significant"] = significance

pivot_table.head(10)

,event_name,metric,test,numerator_tg_1,numerator_tg_2,denominator_tg_1,denominator_tg_2,conversation_rate_tg_1,conversation_rate_tg_2,metric_change,z_stat,p_value,significant
0,add_payment_info,add_payment_info/session,1,1988,2229,45362,45193,0.043825,0.049322,12.542021,-3.924884,0.000087,True
1,add_payment_info,add_payment_info/session,2,2344,2409,50637,50244,0.046290,0.047946,3.576911,-1.240994,0.214608,False
2,add_payment_info,add_payment_info/session,3,3623,3697,70047,70439,0.051722,0.052485,1.474630,-0.643172,0.520112,False
3,add_payment_info,add_payment_info/session,4,3731,3601,105079,105141,0.035507,0.034249,-3.541234,1.571106,0.116158,False
4,add_shipping_info,add_shipping_info/session,1,3034,3221,45362,45193,0.066884,0.071272,6.560481,-2.603571,0.009226,True
5,add_shipping_info,add_shipping_info/session,2,3480,3510,50637,50244,0.068724,0.069859,1.650995,-0.709557,0.477979,False
6,add_shipping_info,add_shipping_info/session,3,5298,5188,70047,70439,0.075635,0.073652,-2.621211,1.413727,0.157442,False
7,add_shipping_info,add_shipping_info/session,4,5128,4956,105079,105141,0.048801,0.047137,-3.411125,1.785795,0.074132,False
8,begin_checkout,begin_checkout/session,1,3784,4021,45362,45193,0.083418,0.088974,6.660587,-2.978783,0.002894,True
9,begin_checkout,begin_checkout/session,2,4262,4313,50637,50244,0.084168,0.085841,1.988164,-0.952898,0.340642,False


###Посилання на excel

https://docs.google.com/spreadsheets/d/11Gzf8J3SOQ3yH-edhOQE_2ECYJCBwXoP/edit?usp=sharing&ouid=116958465139645330281&rtpof=true&sd=true


#AB тести у розрізі континентів, девайсів та каналів

###Обробка даних з метою групування за метриками, тестами та у розрізі континентів, девайсів та каналів


In [8]:
data.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new_account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new_account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new_account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new_account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new_account,1


In [9]:
# Фільтрую дані за необхідними метриками
filtered_data = data[
    (data["event_name"].isin(["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"])) &
    (data["test_group"].isin([1, 2])) &
    (data["test"].isin([1, 2, 3, 4]))]

# Групую відфільтровані дані та рахую суму значень
grouped_data = filtered_data.groupby(["event_name", "test_group", "test", "device", "channel"], as_index=False)["value"].sum()

# Додаю новий стовпчик з назвою метрик
grouped_data["metric"] = grouped_data["event_name"] + "/session"

grouped_data.head()

,event_name,test_group,test,device,channel,value,metric
0,add_payment_info,1,1,desktop,Direct,227,add_payment_info/session
1,add_payment_info,1,1,desktop,Organic Search,334,add_payment_info/session
2,add_payment_info,1,1,desktop,Paid Search,288,add_payment_info/session
3,add_payment_info,1,1,desktop,Social Search,126,add_payment_info/session
4,add_payment_info,1,1,desktop,Undefined,155,add_payment_info/session


###Cтворення зведених таблиць для подальших розрахунків

In [10]:
# Створюю зведену таблицю, щоб згрупувати значення за тестами
pivot_table = grouped_data.pivot(
    index=["event_name", "metric", "test", "device", "channel"],
    columns="test_group",
    values="value").reset_index()

# змінюю назви стовпців
pivot_table.rename(columns={1: "numerator_tg_1", 2: "numerator_tg_2"}, inplace=True)

pivot_table

test_group,event_name,metric,test,device,channel,numerator_tg_1,numerator_tg_2
0,add_payment_info,add_payment_info/session,1,desktop,Direct,227,275
1,add_payment_info,add_payment_info/session,1,desktop,Organic Search,334,271
2,add_payment_info,add_payment_info/session,1,desktop,Paid Search,288,299
3,add_payment_info,add_payment_info/session,1,desktop,Social Search,126,188
4,add_payment_info,add_payment_info/session,1,desktop,Undefined,155,223
...,...,...,...,...,...,...,...
235,new_account,new_account/session,4,tablet,Direct,52,41
236,new_account,new_account/session,4,tablet,Organic Search,80,72
237,new_account,new_account/session,4,tablet,Paid Search,54,58
238,new_account,new_account/session,4,tablet,Social Search,20,17


In [11]:
# фільтрую дані за сесіями
session_data = data[data["event_name"] == "session"]

# групую за необхідними стовпцями та рахую кількість сесій
session_counts = session_data.groupby(["test", "test_group", "device", "channel"], as_index=False)["value"].sum()

# створюю зведену таблицю, щоб згрупувати значення
session_pivot = session_counts.pivot(
    index=["test", "device", "channel"],
    columns="test_group",
    values="value"
).reset_index()

# змінюю назви стовпців
session_pivot.rename(columns={1: "denominator_tg_1", 2: "denominator_tg_2"}, inplace=True)

# доєдную до зведеної таблиці
final_table = pivot_table.merge(session_pivot, on=["test", "device", "channel"], how="left")

final_table.head()

test_group,event_name,metric,test,device,channel,numerator_tg_1,numerator_tg_2,denominator_tg_1,denominator_tg_2
0,add_payment_info,add_payment_info/session,1,desktop,Direct,227,275,6258,6044
1,add_payment_info,add_payment_info/session,1,desktop,Organic Search,334,271,9221,9089
2,add_payment_info,add_payment_info/session,1,desktop,Paid Search,288,299,6843,6959
3,add_payment_info,add_payment_info/session,1,desktop,Social Search,126,188,2232,2355
4,add_payment_info,add_payment_info/session,1,desktop,Undefined,155,223,1913,1970


###Розрахунок необхідних показників та створення фінальної таблиці

In [12]:
# Додаю розрахунок основних показників
final_table["conversation_rate_tg_1"] = final_table["numerator_tg_1"]/final_table["denominator_tg_1"]
final_table["conversation_rate_tg_2"] = final_table["numerator_tg_2"]/final_table["denominator_tg_2"]
final_table["metric_change"] = (final_table["conversation_rate_tg_2"]/final_table["conversation_rate_tg_1"] - 1) * 100

# Створюю пусті списки
z_statistics = []
p_values = []
significance = []

# Роблю ітерацію за рядками
for _, row in final_table.iterrows():
    successes = [row["numerator_tg_1"], row["numerator_tg_2"]]
    sample_sizes = [row["denominator_tg_1"], row["denominator_tg_2"]]

# Створюю z-тест
    z_stat, p_value = proportions_ztest(count=successes, nobs=sample_sizes)

# додаю результати в відповідні списки
    z_statistics.append(z_stat)
    p_values.append(p_value)
    significance.append(p_value < 0.05)

# додаю результати в фінальну таблицю
final_table["z_stat"] = z_statistics
final_table["p_value"] = p_values
final_table["significant"] = significance

final_table.head(10)

test_group,event_name,metric,test,device,channel,numerator_tg_1,numerator_tg_2,denominator_tg_1,denominator_tg_2,conversation_rate_tg_1,conversation_rate_tg_2,metric_change,z_stat,p_value,significant
0,add_payment_info,add_payment_info/session,1,desktop,Direct,227,275,6258,6044,0.036274,0.045500,25.434771,-2.585789,9.715631e-03,True
1,add_payment_info,add_payment_info/session,1,desktop,Organic Search,334,271,9221,9089,0.036222,0.029816,-17.683908,2.424445,1.533182e-02,True
2,add_payment_info,add_payment_info/session,1,desktop,Paid Search,288,299,6843,6959,0.042087,0.042966,2.088872,-0.255901,7.980269e-01,False
3,add_payment_info,add_payment_info/session,1,desktop,Social Search,126,188,2232,2355,0.056452,0.079830,41.413406,-3.133956,1.724665e-03,True
4,add_payment_info,add_payment_info/session,1,desktop,Undefined,155,223,1913,1970,0.081025,0.113198,39.708204,-3.381282,7.214854e-04,True
5,add_payment_info,add_payment_info/session,1,mobile,Direct,157,237,4196,4129,0.037417,0.057399,53.404921,-4.293047,1.762375e-05,True
6,add_payment_info,add_payment_info/session,1,mobile,Organic Search,293,234,6117,6183,0.047899,0.037846,-20.989016,2.752921,5.906614e-03,True
7,add_payment_info,add_payment_info/session,1,mobile,Paid Search,156,211,4675,4592,0.033369,0.045949,37.701158,-3.104781,1.904200e-03,True
8,add_payment_info,add_payment_info/session,1,mobile,Social Search,106,71,1564,1513,0.067775,0.046927,-30.761077,2.483057,1.302604e-02,True
9,add_payment_info,add_payment_info/session,1,mobile,Undefined,98,189,1344,1350,0.072917,0.140000,92.000000,-5.642882,1.672271e-08,True


###Посилання на excel

https://docs.google.com/spreadsheets/d/1-0wz1iHL7fR6Q2D69mlTFGhR2tT_QoTv/edit?usp=sharing&ouid=116958465139645330281&rtpof=true&sd=true

###Візуалізація

Посилання на табло: https://public.tableau.com/views/ABTest_17429319926560/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link